# Potential Speedup imporvements

path generation
* remove any use of ROUND()
* while returning paths, make sure all nodes in the paths are network nodes (no origin or destination nodes..).

subgraph generation
* consoliate + weight

destination discovery
* stop searching for origin nodes past 0.5 search radius
* remove duplicate if statements


update light graph
* shorter list of segments to iterate over
* one add one remove


betweenness
* remove decay calculations from inner loop.
* vectorize betweenness calculation outside path loop
* reduce calls to node_gdf.at[] by doing them when thwey're first possible, instead of inside path loop
* vectorize destination probability calculations



path generation
* return paths as edges
* remove len(neighbors) = 1 check, causes excessive calls to neighbors.
* consolidate edge information retreval from graph.

betweenness
* eliminate edge list construction inside loop.

parallel Betweenness:
* Implement a queue for parallel processing
* better reporting from quue progress

Betweenness
* update to accomidate queue feeding
* implement controls for starting a job only when there is enough memory...
* selete large variable prior to next job, so save for memory when waiting for memory to free up

path generation:
* remove edge sets, sacrificing time performance for memory savings
* slightly more optimized scope neighbor, use set operations to filter neighbors in scope, instead of list comprehension with checking..
* experament with dequeue for cheaper path appends...
* significiant memory redunction by eliminating need to store "targets remaining", this might lead to reduction in cpu effeciency?






# For next week:
* Send rounaq wide table of counts
* Once recieved, from lui, start a new digitization process in the new network
* look at th eparks file situation

# Functionality List 3
* set up a slideshow/diagram to show the relationship between different compoonents of the library.
* Set up a notebook to go over estimating flows from one origin to one destination, Document all relevant settings
* show how this generalizes to the pairings.csv, iterating over moodel settings, iterating over scenarios
* Make a finalized github commit including testing script, betweenness flow notebook. 
* Deploy to workstation.
* given the simulation results, and the censor locations, find a way to calibrate counts and generate beta coeffecients (Beirut, SOmerville, NYC)
* enable Jupyter notebook server access from workstation.
* Figure out a way to do the "Cities" dropbox folder, people can add data through dropbox, run simulations through Jupyter Server, get results through Dropbox

## Map Cleaning
https://www.dropbox.com/scl/fo/d6ugcyt3aanfb3yk4n9va/h?dl=0&rlkey=1c2giilvzoyygcxrlry7fscr9



## Needed compatibility upgrades:
* The query_bulk() method of the spatial index .sindex property is deprecated in favor of query() (#2823).
* Make surr there is no direct use of pyGEOS (currently used in node-edge creation if tolerance=0) [ If you use PyGEOS directly and access an array of PyGEOS geometries using GeoSeries.values.data, you will need to make some changes to avoid code breakage.]  , a simple replacement of pygeos with shapely together with a change of gdf.geometry.values.data to gdf.geometry.values or analogous gdf.geometry.array should work


## instaling an environment
```
Run Powershell as administrator

conda create --name=madina_0_0_2 python
conda activate madina_0_0_2
conda config --env --add channels conda-forge
conda config --env --set channel_priority strict
conda install -f mkl
conda install python=3 geopandas
conda install pydeck
conda install pyogrio
conda install rtrees
```

In [1]:
%config Completer.use_jedi = False

import sys
sys.path.append('../')

from madina.zonal.zonal import Zonal
from madina.una.betweenness import parallel_betweenness, paralell_betweenness_exposure, betweenness_exposure


import pandas as pd
import geopandas as gpd
import numpy as np
import time 



print (f"{'test name':30s} | {'madina_flow_sum':15s} | {'Rhino_flow_sum':15s} | {'sum_diff':8s} | {'sum_smlr_%':10s} | {'sle_mean':8s} | {'sle_median':10s} | {'sle_max':8s} | {'sle>0.1%':8s} | {'sle>1.0%':8s} | {'sle>3.0%':8s} | {'sle>5.0%':8s} | {'time_spent':10s}")
#for test_case in os.listdir("Test Cases"):
for test_case in ['Harvard Square', 'Somerville']:   # 'Harvard Square',
    start = time.time()
    # TODO: Check OS compatibility, ensure this is compatible with Unix systems..
    test_case_folder = "Test Cases" + "\\" + test_case + "\\"
    test_config = pd.read_csv(test_case_folder + "test_configs.csv")
    test_flows =  pd.read_csv(test_case_folder + "test_flows.csv")

    if test_case == 'Somerville':
        ready_tests = test_config.index[0:3]
    else:
        ready_tests = test_config.index

    for test_idx in ready_tests:

        harvard_square = Zonal()

        harvard_square.load_layer(
            layer_name='streets',
            file_path=  test_case_folder + test_config.at[test_idx, 'Network_File']
            )

        harvard_square.load_layer(
            layer_name=test_config.at[test_idx, 'Origin_Name'],
            file_path= test_case_folder + test_config.at[test_idx, 'Origin_File']
            )

        harvard_square.load_layer(
            layer_name=test_config.at[test_idx, 'Destination_Name'],
            file_path= test_case_folder + test_config.at[test_idx, 'Destination_File']
            )
        

        harvard_square.create_street_network(
            source_layer='streets', 
            discard_redundant_edges=True,
            split_redundant_edges=False,
            node_snapping_tolerance=0.1,  # TODO: check for sensitivity... pick one as default snapping.
            weight_attribute=test_config.at[test_idx, 'Network_Cost'] if test_config.at[test_idx, 'Network_Cost'] != "Geometric" else None
        )

        #print (f"{harvard_square.network.edges.shape = }")
        #print (f"{harvard_square.network.nodes.shape = }")
        #harvard_square.network.nodes,  harvard_square.network.edges = _discard_redundant_edges(harvard_square.network.nodes, harvard_square.network.edges)
        #print (f"{harvard_square.network.edges.shape = }")
        #print (f"{harvard_square.network.nodes.shape = }")

        harvard_square.insert_node(
            layer_name=test_config.at[test_idx, 'Origin_Name'], 
            label='origin', 
            weight_attribute=test_config.at[test_idx, 'Origin_Weight'] if test_config.at[test_idx, 'Origin_Weight'] != "Count" else None
        )
        harvard_square.insert_node(
            layer_name=test_config.at[test_idx, 'Destination_Name'], 
            label='destination', 
            weight_attribute=test_config.at[test_idx, 'Destination_Weight'] if test_config.at[test_idx, 'Destination_Weight'] != "Count" else None
        )

        harvard_square.create_graph()

        node_gdf = harvard_square.network.nodes
        origin_gdf = node_gdf[node_gdf['type'] == 'origin']

        harvard_square.network.nodes["original_weight"] = harvard_square.network.nodes["weight"]

        harvard_square.network.turn_penalty_amount = test_config.at[test_idx, 'Turn_Penalty']
        harvard_square.network.turn_threshold_degree = test_config.at[test_idx, 'Turn_Threshold']

        if test_config.at[test_idx, 'Elastic_Weights']:
            '''
            harvard_square.network.nodes["weight"] = harvard_square.network.nodes["original_weight"]
            get_elastic_weight(
                harvard_square.network,
                search_radius=test_config.at[test_idx, 'Radius'],
                detour_ratio=test_config.at[test_idx, 'Detour'],
                beta=test_config.at[test_idx, 'Beta'],
                decay=True, #test_config.at[test_idx, 'Decay'],
                #turn_penalty=test_config.at[test_idx, 'Turns'],
                turn_penalty=False,
            )
            for o_idx in origin_gdf.index:
                harvard_square.network.nodes.at[o_idx, 'weight'] =  harvard_square.network.nodes.at[o_idx, 'elastic_weight']
            '''
            continue
        '''
        return_dict = parallel_betweenness(
            harvard_square.network,
            search_radius=test_config.at[test_idx, 'Radius'],
            detour_ratio=test_config.at[test_idx, 'Detour'],
            decay=test_config.at[test_idx, 'Decay'], #if test['Elastic weights'] else True,
            decay_method=test_config.at[test_idx, 'Decay_Mode'],  # "power", "exponent"
            beta=test_config.at[test_idx, 'Beta'],
            path_detour_penalty='equal', # "equal",  # "power", "exponent", "equal"
            origin_weights=False if type(test_config.at[test_idx, 'Origin_Weight']) != str else True,
            closest_destination=test_config.at[test_idx, 'Closest_Destination'],
            destination_weights=False if type(test_config.at[test_idx, 'Destination_Weight']) != str  else True,    #or (test['Elastic weights'])
            # perceived_distance=False,
            num_cores=6,
            light_graph=True,
            turn_penalty=test_config.at[test_idx, 'Turns'],
        )
        
        '''
        return_dict = paralell_betweenness_exposure(
            harvard_square,
            search_radius=test_config.at[test_idx,'Radius'],
            detour_ratio=test_config.at[test_idx,'Detour'],
            decay=False if test_config.at[test_idx,'Elastic_Weights'] else test_config.at[test_idx,'Decay'],  # elastic weight already reduces origin weight factoring in decay. if this pairing uses elastic weights, don't decay again,
            decay_method=test_config.at[test_idx,'Decay_Mode'],
            beta=test_config.at[test_idx,'Beta'],
            num_cores=3,
            path_detour_penalty='equal', # "power" | "exponent" | "equal"
            closest_destination=test_config.at[test_idx,'Closest_Destination'],
            elastic_weight=test_config.at[test_idx,'Elastic_Weights'],
            turn_penalty=test_config.at[test_idx,'Turns'],
            path_exposure_attribute=None,
            return_path_record=False, 
            destniation_cap=None
        )

        



        simulated_sum_of_flow = return_dict['edge_gdf']['betweenness'].sum()
        test_flow = test_flows[test_config.at[test_idx, 'Flow_Name']].sum()


        ## create segment level comparison
        # creating connector lines

        import shapely.geometry as geo
        simulated_betweenness = return_dict['edge_gdf'][['betweenness', 'parent_street_id']].rename(columns={"betweenness": "simulated_betweenness"}).drop_duplicates(subset=['parent_street_id']).set_index("parent_street_id")
        simulated_betweenness = harvard_square.layers["streets"].gdf[["geometry", "__GUID"]].join(simulated_betweenness).set_index("__GUID")

        test_name = test_config.at[test_idx, 'Flow_Name']
        test_betweenness = test_flows[['__GUID', test_name]].set_index("__GUID").rename(columns = {test_name: "test_flow"})


        comparison = simulated_betweenness.join(test_betweenness)
        comparison["difference"] = comparison["simulated_betweenness"] - comparison["test_flow"]
        comparison["difference_pct"] = abs(comparison["difference"]) / comparison["simulated_betweenness"] *100
        # segment level error
        sle = comparison[~comparison["difference_pct"].isna()]['difference_pct']

        
        node_gdf = harvard_square.network.nodes
        edge_gdf = harvard_square.network.edges


        origin_nodes = node_gdf[node_gdf['type'] == 'origin']
        '''
        origin_layer = harvard_square.layers[test_config.at[test_idx, 'Origin_Name']].gdf
        origin_joined = origin_layer.join(origin_nodes.set_index('source_id'),lsuffix='_origin')
        origin_joined['connector_line'] = origin_joined.apply(lambda x:geo.LineString([x['geometry'], x["geometry_origin"]]), axis=1)
        origin_joined["geometry"] = origin_joined['connector_line']


        destination_layer = harvard_square.layers[test_config.at[test_idx, 'Destination_Name']].gdf
        destination_nodes = node_gdf[node_gdf['type'] == 'destination']
        destination_joined = destination_layer.join(destination_nodes.set_index('source_id'),lsuffix='_destination')
        destination_joined['connector_line'] = destination_joined.apply(lambda x:geo.LineString([x['geometry'], x["geometry_destination"]]), axis=1)
        destination_joined["geometry"] = destination_joined['connector_line']


        #print (f"{origin_nodes.shape = }\t{destination_nodes.shape = }")

        streets = harvard_square.layers["streets"].gdf 
        network_file = gpd.read_file(test_case_folder + test_config.at[test_idx, 'Network_File'], engine='pyogrio')


        flow_difference = comparison[
            ((comparison['test_flow' ] > 0) & (comparison['simulated_betweenness'] == 0)) | 
            ((comparison['test_flow' ] == 0) & (comparison['simulated_betweenness'] > 0))
        ]
        harvard_square.create_map(
            [
                #{'gdf': streets[streets[test_config.at[test_idx, 'Network_Cost']] > 0], 'color': [255, 255, 255], 'text': test_config.at[test_idx, 'Network_Cost']},
                {'gdf': streets, 'color': [100, 100, 100], 'opacity': 0.1},
                {'gdf': edge_gdf[edge_gdf['betweenness'] > 0], 'color': ['125, 125, 0'], 'text': 'betweenness', 'opacity': 0.2},
                #{'gdf': comparison[abs(comparison["difference"]) > 0.01], 'color_by_attribute': 'difference', 'color_method': 'gradient', 'text': 'difference'},
                {'gdf': comparison[(comparison["difference_pct"] >= 0.1) & (comparison["difference_pct"] < 100)], 'color_by_attribute': 'difference_pct', 'color_method': 'gradient', 'text': 'difference_pct'},
                {'gdf': edge_gdf[edge_gdf['snapped'] == True], 'color': [255, 0, 255], 'opacity': 0.2},
                {'gdf': network_file[network_file["geometry"].geom_type == 'Polygon'], 'color': [125, 0, 125], 'text': '__GUID'},
                {'gdf': flow_difference, 'color': [255, 255, 0] , 'text': 'difference'},        
                #{'gdf': comparison[comparison['difference'] != 0], 'color_by_attribute': 'difference', 'color_method': 'gradient', 'text': 'difference', 'opacity':  0.1},
                {'gdf': origin_layer, 'color': [0, 0, 255]},
                {'gdf': origin_joined[['geometry']], 'color': [0, 0, 255]},
                {'gdf': destination_layer, 'color': [255, 0, 0]},
                {'gdf': destination_joined[['geometry']], 'color': [255, 0, 0]},
                {'gdf': harvard_square.network.nodes[['geometry', 'type', 'weight']].reset_index(), 'color': [255, 0, 255], 'text': 'id'},
            ],
            save_as="Test Cases\\" + test_case + '\\'  + test_name + "._difference_map.html"
        )
        '''
        
        #print (test_config.loc[test_idx])
        print (f"{test_config.at[test_idx, 'Flow_Name'][:30]:30s} | {simulated_sum_of_flow:15.2f} | {test_flow:15.2f} | {simulated_sum_of_flow - test_flow:8.2f} | {1-(simulated_sum_of_flow - test_flow)/ test_flow:10.2%} | {sle.mean():7.4f}% | {sle.median():9.4f}% | {sle.max():7.4f}% | {sle[sle > 0.1].count():8} | {sle[sle > 1.0].count():8} | {sle[sle > 3.0].count():8} | {sle[sle > 5.0].count():8} | {time.time() - start: 10.5f}")
        #print ("DOne Case...")
        #break
    #break

test name                      | madina_flow_sum | Rhino_flow_sum  | sum_diff | sum_smlr_% | sle_mean | sle_median | sle_max  | sle>0.1% | sle>1.0% | sle>3.0% | sle>5.0% | time_spent
_Betweenness1                  |          702.07 |          702.07 |    -0.00 |    100.00% |  0.0000% |    0.0000% |  0.0000% |        0 |        0 |        0 |        0 |    3.55432
_Betweenness2                  |        19025.63 |        19025.63 |     0.00 |    100.00% |  0.0000% |    0.0000% |  0.0000% |        0 |        0 |        0 |        0 |    6.61948
_Betweenness3                  |        19214.00 |        19214.00 |     0.00 |    100.00% |  0.0000% |    0.0000% |  0.0000% |        0 |        0 |        0 |        0 |   10.24690
_Betweenness4                  |        20853.18 |        20853.18 |    -0.00 |    100.00% |  0.0000% |    0.0000% |  0.0000% |        0 |        0 |        0 |        0 |   13.82649
Somerville_Bus_Subway_Geometri |          700.29 |          713.14 |   -12.84 |    10

In [ ]:
# keeping track of visited destinations
test name                      | madina_flow_sum | Rhino_flow_sum  | sum_diff | sum_smlr_% | sle_mean | sle_median | sle_max  | sle>0.1% | sle>1.0% | sle>3.0% | sle>5.0% | time_spent
_Betweenness1                  |          702.07 |          702.07 |    -0.00 |    100.00% |  0.0000% |    0.0000% |  0.0000% |        0 |        0 |        0 |        0 |    4.27237
_Betweenness2                  |        19025.63 |        19025.63 |     0.00 |    100.00% |  0.0000% |    0.0000% |  0.0000% |        0 |        0 |        0 |        0 |    7.92910
_Betweenness3                  |        19214.00 |        19214.00 |     0.00 |    100.00% |  0.0000% |    0.0000% |  0.0000% |        0 |        0 |        0 |        0 |   11.60755
_Betweenness4                  |        20853.18 |        20853.18 |    -0.00 |    100.00% |  0.0000% |    0.0000% |  0.0000% |        0 |        0 |        0 |        0 |   15.20730
Somerville_Bus_Subway_Geometri |          700.29 |          713.14 |   -12.84 |    101.80% |  2.3488% |    2.2893% |  5.0852% |      266 |      232 |       96 |        2 |   12.55345
Somerville_Bus_Subway          |          730.95 |          743.95 |   -13.00 |    101.75% |  2.4117% |    2.2917% |  5.1912% |      285 |      251 |      103 |        8 |   25.87746
Somerville_Homes_Subway        |       368786.93 |       389383.31 | -20596.38 |    105.29% |     inf% |    6.9536% |     inf% |     2640 |     2565 |     2355 |     1966 |  277.37418


In [2]:
import multiprocessing as mp
def profile_new():
    origin_gdf = origin_nodes
    num_cores = 1
    with mp.Manager() as manager:
        #create aNDfill queue
        origin_queue = manager.Queue()
        for o_idx in origin_gdf.index[:]:
            origin_queue.put(o_idx)
        for core_index in range(num_cores):
            origin_queue.put("done")


        return_dict = betweenness_exposure(
            harvard_square,
            core_index=1,
            origin_queue=origin_queue,
            #origins=origin_gdf.iloc[:1000],
            search_radius=test_config.at[test_idx,'Radius'],
            detour_ratio=test_config.at[test_idx,'Detour'],
            decay=False if test_config.at[test_idx,'Elastic_Weights'] else test_config.at[test_idx,'Decay'],
            beta=test_config.at[test_idx,'Beta'],
            decay_method=test_config.at[test_idx,'Decay_Mode'],
            path_detour_penalty='equal',
            closest_destination=test_config.at[test_idx,'Closest_Destination'],
            elastic_weight=test_config.at[test_idx,'Elastic_Weights'],
            turn_penalty=test_config.at[test_idx,'Turns'],
            path_exposure_attribute=None,
            return_path_record=False, 
            destniation_cap=None
    )
    return return_dict
x = profile_new()

DOne 100
DOne 200
DOne 300
DOne 400
DOne 500
DOne 600
DOne 700
DOne 800
DOne 900
DOne 1000
DOne 1100
DOne 1200
DOne 1300
DOne 1400
DOne 1500
DOne 1600
DOne 1700
DOne 1800
DOne 1900
DOne 2000
DOne 2100
DOne 2200
DOne 2300
DOne 2400
DOne 2500
DOne 2600
DOne 2700
DOne 2800


KeyboardInterrupt: 